In [1]:
import os
import json
from pathlib import Path

import pandas as pd

from neuralhydrology.nh_run import eval_run

In [2]:
# load allowed GPU ids
f = open("gpu.json")
gpus = json.load(f)
f.close()

In [3]:
epoch = 20
epoch_str = str(epoch) if epoch / 100 > 1 else f"0{epoch}"

In [4]:
run_dir = Path(f"./runs/{sorted(os.listdir('./runs'))[-1]}")

In [5]:
def eval_results(period: str, epoch: str = "010") -> pd.DataFrame:
    df = pd.read_csv(run_dir / f"{period}" / f"model_epoch{epoch}" / f"{period}_metrics.csv", dtype={'basin': str})
    df = df.set_index('basin')

    # Compute the median NSE from all basins, where discharge observations are available for that period
    print(f"Median NSE of the {period} period {df['NSE'].median():.3f}")
    print(f"Mean NSE of the {period} period {df['NSE'].mean():.3f}")
    
    return df

In [6]:
eval_run(run_dir, period="train", epoch=epoch, gpu=gpus[0])
df_train = eval_results("train", epoch=epoch_str)

# Evaluation:  19%|█▉        | 8/42 [00:13<00:51,  1.53s/it]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  40%|████      | 17/42 [00:24<00:35,  1.43s/it]

Basin 13002 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  43%|████▎     | 18/42 [00:26<00:35,  1.48s/it]

...

# Evaluation:  67%|██████▋   | 28/42 [00:46<00:30,  2.19s/it]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Evaluation:  83%|████████▎ | 35/42 [00:54<00:07,  1.09s/it]

Basin 19243 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation: 100%|██████████| 42/42 [01:05<00:00,  1.55s/it]
Median NSE of the train period -64936.520
Mean NSE of the train period -269724.880


In [7]:
eval_run(run_dir, period="validation", epoch=epoch, gpu=gpus[0])
df_val = eval_results("validation", epoch=epoch_str)

# Validation:  10%|▉         | 4/42 [00:00<00:09,  4.13it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Validation:  19%|█▉        | 8/42 [00:02<00:09,  3.77it/s]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


# Validation:  45%|████▌     | 19/42 [00:05<00:05,  4.06it/s]

...

# Validation:  60%|█████▉    | 25/42 [00:06<00:04,  3.79it/s]

Basin 13115 All simulated values are NaN, thus metrics will be NaN, too.


# Validation:  67%|██████▋   | 28/42 [00:07<00:04,  3.29it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Validation: 100%|██████████| 42/42 [00:11<00:00,  3.60it/s]
Median NSE of the validation period -60674.793
Mean NSE of the validation period -505097.116


In [8]:
eval_run(run_dir, period="test", epoch=epoch, gpu=gpus[0])
df_test = eval_results("test", epoch=epoch_str)

# Evaluation:   0%|          | 0/42 [00:00<?, ?it/s]

# Evaluation:  10%|▉         | 4/42 [00:00<00:08,  4.68it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Evaluation:  19%|█▉        | 8/42 [00:02<00:08,  3.87it/s]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


...

# Evaluation:  62%|██████▏   | 26/42 [00:07<00:04,  3.52it/s]

Basin 13128 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  67%|██████▋   | 28/42 [00:07<00:04,  3.38it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Evaluation: 100%|██████████| 42/42 [00:11<00:00,  3.67it/s]
Median NSE of the test period -267570.570
Mean NSE of the test period -337420976.639


In [9]:
df_train = df_train.rename(columns={"NSE": "NSE_train", "KGE": "KGE_train"})
df_val = df_val.rename(columns={"NSE": "NSE_val", "KGE": "KGE_val"})
df_test = df_test.rename(columns={"NSE": "NSE_test", "KGE": "KGE_test"})

In [10]:
df_tmp = pd.merge(df_train, df_val, left_index=True, right_index=True)
df = pd.merge(df_tmp, df_test, left_index=True, right_index=True)
df.head()

,NSE_train,KGE_train,NSE_val,KGE_val,NSE_test,KGE_test
basin,,,,,,
11001,-19574.519531,-159.382336,-14028.236328,-124.936326,-53263.722656,-186.224799
11068,-54114.101562,-167.558446,-60674.792969,-193.622422,-80188.343750,-163.476423
11126,-1138.022949,-35.941900,-1181.884766,-34.458181,-2102.470947,-39.715560
11129,-707.492615,-30.040096,-848.496338,-31.677659,-1340.977661,-38.917803
11163,-125.740479,-9.905895,NaN,NaN,NaN,NaN


File will be saved in the current runs directory

In [11]:
df.to_csv(f"{str(run_dir)}/eval.csv")